# Horizon3D Class - ODBind Python Bindings

In [1]:
odpy_path = '/home/wayne/Work/WMSeismicSolutions/dGB/Development/Build/bin/od7.0/bin/python'
data_root = '/mnt/Data/seismic/ODData'
import os
import sys

In [2]:
sys.path.insert(0,odpy_path)
os.environ["DTECT_DATA"] = data_root
import odbind as odb

The Horizon3D class provides access to 3D horizons in an OpendTect project/survey. A Survey object is required for context.

In [3]:
f3demo = odb.Survey(data_root, 'F3_Demo_2020')

In [4]:
mfs4 = odb.Horizon3D(f3demo, 'Demo 1 --> MFS4')
mfs4.info()

{'Name': 'Demo 1 --> MFS4',
 'NrAttribute': 3,
 'Z Range': [0.528257012367249, 1.09793794155121],
 'Inl Range': [100, 750, 1],
 'Crl Range': [300, 1250, 1]}

In [5]:
mfs4.attribnames

['SD_24Hz[-8,24ms]', 'SD_44Hz[-8,24ms]', 'SD_64Hz[-8,24ms]']

## Static methods
A number of methods are provided to get information either for all or a selected number of 3D horizons in the user provided survey.

In [6]:
hors = odb.Horizon3D.names(f3demo)
hors

['Demo 7 --> shallow',
 'Demo 6 --> FS8',
 'Demo 5 --> FS7',
 'Demo 4 --> Truncation',
 'Demo 3  --> Top Foresets',
 'Demo 2 --> FS6',
 'Demo 1 --> MFS4',
 'Demo 0 --> FS4',
 'Trim_D6 --> FS8',
 'Trim_D5 --> FS7',
 'Trim_D4 --> Truncation',
 'Trim_D0 --> FS4']

In [7]:
odb.Horizon3D.infos(f3demo, hors[6:8])

{'Name': ['Demo 1 --> MFS4', 'Demo 0 --> FS4'],
 'NrAttribute': [3, 1],
 'Z Range': [[0.528257012367249, 1.09793794155121],
  [0.574454009532928, 1.126086950302124]],
 'Inl Range': [[100, 750, 1], [100, 750, 1]],
 'Crl Range': [[300, 1250, 1], [300, 1250, 1]]}

In [8]:
odb.Horizon3D.features(f3demo, hors[6:8])

'{"type":"FeatureCollection","features":[{"type":"Feature","properties":{"Name":"Demo 1 --> MFS4","NrAttribute":3,"Z Range":[0.528257012367249,1.09793794155121],"Inl Range":[100,750,1],"Crl Range":[300,1250,1]},"geometry":{"type":"Polygon","coordinates":[[4.644803351594548,54.796120223953054],[4.64367610071001,54.94212694984212],[5.014355742545435,54.942512524559866],[5.01414540463039,54.796514396823831],[4.644803351594548,54.796120223953054]]}},{"type":"Feature","properties":{"Name":"Demo 0 --> FS4","NrAttribute":1,"Z Range":[0.574454009532928,1.126086950302124],"Inl Range":[100,750,1],"Crl Range":[300,1250,1]},"geometry":{"type":"Polygon","coordinates":[[4.644803351594548,54.796120223953054],[4.64367610071001,54.94212694984212],[5.014355742545435,54.942512524559866],[5.01414540463039,54.796514396823831],[4.644803351594548,54.796120223953054]]}}]}'